#Загрузка gt и создание sub

  sub - submission файл
  
  gt - grounded true файл

In [1]:
import pandas as pd

In [2]:
gt = pd.read_excel('data/train_data/train_data_markup.xlsx', keep_default_na=False)

In [3]:
gt.head(5)

,Number,Name,Differences,Description,Complience Level
0,8800,Receiving Call Notifications,SSTS misses the following content:\n The outpu...,HMX description:\nThe driver can accept or rej...,LC
1,31523,Adding Internet Radio to Favorites List,UC misses the following content:\nUsers can se...,Users can add or remove songs from favorites a...,LC
2,25957,Mute/unmute the FM Radio playback,There is no soft button icon and description i...,The user clicks on the radio and clicks on the...,PC
3,26160,Filtering the internet radio stations,ssts hasn't info about this,-,NA
4,11467,Revoke access to the vehicle from a driver or ...,-,The driver opens the ATOM application on his i...,FC


In [4]:
gt['Complience Level'].unique()

array(['LC', 'PC', 'NA', 'FC', 'NC'], dtype=object)

In [5]:
gt.to_csv('data/grounded_true.csv', index = False)

In [6]:
import numpy as np

#Симуляция реального сабмита
def simul_sub(df, columns_to_swap_modify):
    len_base_df = len(df)

    # Процент строк для удаления и дублирования
    fraction_to_remove = 0.1
    fraction_to_duplicate = 0.1

    num_to_remove = int(len_base_df * fraction_to_remove)
    num_to_duplicate = int(len_base_df * fraction_to_duplicate)

    # Удаление случайных строк
    rows_to_remove = df.sample(num_to_remove).index
    data_dropped = df.drop(rows_to_remove)

    # Дублирование случайных строк
    rows_to_duplicate = data_dropped.sample(num_to_duplicate)
    data_final = pd.concat([data_dropped, rows_to_duplicate])
    data_final = data_final.reset_index(drop=True)

    # Замена значений с вероятностью swap_chance
    swap_chance=0.4

    for idx in data_final.index:
        for column in columns_to_swap_modify:
            if np.random.rand() < swap_chance:
                # Выбираем случайное значение из того же столбца
                random_row = data_final.sample(1).iloc[0]
                data_final.at[idx, column] = random_row[column]

    return data_final

In [7]:
#Симуляция реального сабмита
sub = simul_sub(gt.copy(), ['Complience Level'])

#Или с помощью моды
#sub = gt.copy()
#sub['Complience Level'] = gt['Complience Level'].mode()[0]

In [8]:
sub.to_csv('data/submission.csv', index = False)

In [9]:
sub.head(5)

,Number,Name,Differences,Description,Complience Level
0,8800,Receiving Call Notifications,SSTS misses the following content:\n The outpu...,HMX description:\nThe driver can accept or rej...,LC
1,31523,Adding Internet Radio to Favorites List,UC misses the following content:\nUsers can se...,Users can add or remove songs from favorites a...,LC
2,25957,Mute/unmute the FM Radio playback,There is no soft button icon and description i...,The user clicks on the radio and clicks on the...,PC
3,26160,Filtering the internet radio stations,ssts hasn't info about this,-,NA
4,11467,Revoke access to the vehicle from a driver or ...,-,The driver opens the ATOM application on his i...,FC


#Метрика

score = max(1.5 - mse (по числовому варианту столбца 'Complience Level'), 0)/1.5


In [10]:
from sklearn.metrics import mean_squared_error
def calc_score(gt, sub):
  sub = sub.drop_duplicates(subset='Number', keep='last')

  mapping = {'FC': 1, 'LC': 2, 'PC': 3, 'NC': 4, 'NA': 5}
  gt['категории_числа'] = gt['Complience Level'].map(mapping)
  sub['категории_числа'] = sub['Complience Level'].map(mapping)


  merge_df = pd.merge(gt, sub, on='Number', how='left')
  merge_df['категории_числа_y'] = merge_df['категории_числа_y'].fillna(mapping['NA'])
  mse = mean_squared_error(merge_df['категории_числа_x'], merge_df['категории_числа_y'])

  score = max(0, 1.5-mse)/1.5

  return score

In [11]:
score = calc_score(gt.copy(), sub.copy())

/tmp/ipykernel_26109/2736361996.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['категории_числа'] = sub['Complience Level'].map(mapping)


In [12]:
score

np.float64(0.4444444444444444)